## 작업 Flow
1. 논문에서 가장 성능이 좋은 bertalign으로 얼라인(align) 선택
2. 수집한 데이터를 문장 기준으로 얼라인(align)하기
3. json 형태의 데이터를 input에 맞게 조정
4. 얼라인(align)
  - sample로 영어-프랑스어 얼라인(align) 결과 출력


In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!git clone https://github.com/bfsujason/bertalign

Cloning into 'bertalign'...
remote: Enumerating objects: 1157, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 1157 (delta 18), reused 15 (delta 15), pack-reused 1130
Receiving objects: 100% (1157/1157), 303.20 MiB | 25.21 MiB/s, done.
Resolving deltas: 100% (425/425), done.


In [4]:
cd bertalign

/content/bertalign


In [6]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
from bertalign import Bertalign

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/391 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

In [28]:
import json

# JSON 파일 불러오기
with open('/content/drive/MyDrive/베링랩_윤승현/베링랩/data.json', 'r') as file:
    data = json.load(file)

In [35]:
# language_codes = 'en', 'es-419', 'fr', 'id', 'it', 'pl', 'pt-br', 'vi', 'tr', 'ru', 'he', 'ar', 'fa', 'hi', 'bn', 'th', 'zh-cn', 'zh-tw', 'ja', 'ko'

def extract_text_from_data(data, language_code):
    text_string = ""

    for link, context in data[language_code].items():
        # 'context'는 이미 리스트 형태입니다.
        text_string += ' '.join(context) + ' '

    return text_string.strip()

# 'en' 언어에 대한 텍스트 추출
en_text = extract_text_from_data(data, 'en')
print("English Text:")
print(en_text)

# 'fr' 언어에 대한 텍스트 추출
fr_text = extract_text_from_data(data, 'fr')
print("\franch Text:")
print(fr_text)


English Text:
TensorFlow is tested and supported on the following 64-bit systems: Install TensorFlow with Python's pip package manager. Official packages available for Ubuntu, Windows, and macOS. 
  The TensorFlow
  Docker images are already configured to run TensorFlow. A
  Docker container runs in a
  virtual environment and is the easiest way to set up GPU support.
 
  No install necessary—run the TensorFlow tutorials directly in
  the browser with Colaboratory, a Google research project created to help disseminate
  machine learning education and research. It's a Jupyter notebook environment that requires
  no setup to use and runs entirely in the cloud.
  Read the blog post.
 Except as otherwise noted, the content of this page is licensed under the Creative Commons Attribution 4.0 License, and code samples are licensed under the Apache 2.0 License. For details, see the Google Developers Site Policies. Java is a registered trademark of Oracle and/or its affiliates. Last updated 202

In [36]:
src = en_text
tgt = fr_text

In [37]:
aligner = Bertalign(src, tgt)
aligner.align_sents()

Source language: English, Number of sentences: 516
Target language: French, Number of sentences: 500
Embedding source and target text using LaBSE ...
Performing first-step alignment ...
Performing second-step alignment ...
Finished! Successfully aligning 516 English sentences to 500 French sentences



In [38]:
aligner.print_sents()

TensorFlow is tested and supported on the following 64-bit systems: Install TensorFlow with Python's pip package manager.
TensorFlow a été testé et est compatible avec les systèmes 64 bits suivants : Installez TensorFlow avec le gestionnaire de packages pip de Python.

Official packages available for Ubuntu, Windows, and macOS.
Packages officiels disponibles pour Ubuntu, Windows et macOS.


Consultez le guide des GPU pour connaître les cartes compatibles CUDA®.

The TensorFlow Docker images are already configured to run TensorFlow.
Les images Docker TensorFlow sont déjà configurées pour l'exécution de TensorFlow.

A Docker container runs in a virtual environment and is the easiest way to set up GPU support.
Un conteneur Docker s'exécute dans un environnement virtuel et constitue le moyen le plus simple de configurer la compatibilité avec les GPU.

No install necessary—run the TensorFlow tutorials directly in the browser with Colaboratory, a Google research project created to help disse